# Creating the Initial Commitment Transaction

In this section, we'll build a lightning channel initial commitment transaction from scratch using Python. We'll walk through each part of the transaction — how it's constructed and signed. The process will be tested using Bitcoin Core in regtest mode.

## Setup

For this notebook, we’ll use the basepoints derivated in the `chapter 0 - lightning node keys derivation`, and the funding transaction created in the `chapter 1 - channel funding transaction`.

In [10]:
%run "../chapter 1 - channel funding transaction/funding transaction.ipynb"

Alice Per Commitment Seed 34b581ec20bf2c6cae3d4d4dcbfddc8a3727a1e9a57c55f3520e770607898c06
Bob Per Commitment Seed 89c994b3ddad4698acee71e42d8bcace48eea739caaba371eb110e77663ec56d
Alice Payment BasePoint:  025f892a06124391e2f38ce35d943cdc09f63e203330dbd9cb6113a903e0738458
Bob Payment BasePoint:  02f98efd3f2b2fbe7bd83c419f5f64f8280798b8a9175fdb77c0091bbb95c79506
To obscure commitment number 0xb433fd43a66f
Alice funding pubkey: 427602908d25c6fcbdce87b5d0066b31db0bf744316df73e8fd7b42e3b4f500c
Alice funding privkey: a11cc9c57675ec0f4bcacd3faffe75ba3bb8335b205401a9f782b52c9b1d3140
Alice funding address: bcrt1pgfmq9yydyhr0e0wws76aqpntx8dsha6yx9klw050676zuw602qxq5h0ddm
Bob funding pubkey: ed869275d191be4a4463799c2c4afa61f41987ae9159fa396336d5c89a80104f
Bob funding privkey: dfdd10125360efaf34e3564c4fde8d8438dcb0e17c2f9de6e5edf5e2350998f0
Bob funding address: bcrt1pakrfyaw3jxly53rr0xwzcjh6v86pnpawj9vl5wtrxm2u3x5qzp8sjmvef3
Transaction creating Alice UTXO: 93e6f5dc4b4577b3bb34d27cf471e4a9d65e6e7

In [11]:
from functions.test_framework.script import Tapbranch, TapTree, TapLeaf, CScript, TaprootSignatureHash, OP_CHECKSIG, OP_CHECKSIGVERIFY, OP_CHECKSEQUENCEVERIFY, OP_DROP, OP_16, OP_1, OP_SIZE, OP_EQUALVERIFY, OP_HASH160
from functions.test_framework.messages import CTxInWitness, ser_string
TAPSCRIPT_VER = bytes([0xc0])
# A Taproot NUMS key is a fixed secp256k1 public key with no known private key. Used as the internal key to disable key-path spending, forcing script-path-only spends.
NUMS = ECPubKey().set(bytes.fromhex("02dca094751109d0bd055d03565874e8276dd53e926b44e3bd1bb6bf4bc130a279"))


At this point, no HTLCs have been added yet, which makes the initial commitment transaction simpler. Only after Alice has a fully signed initial commitment transaction she will broadcast the funding transaction, this serves as a guarantee for Alice (the funder) that she can reclaim her sats back if anything goes wrong.

### The Unsigned Transaction

### The Input

The input is the channel funding transaction.

In [12]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(funding_channel_txid))[::-1]
funding_channel_index = 0
index = funding_channel_index.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# sequence: upper 8 bits are 0x80, lower 24 bits are the upper 24 bits of the obscured commitment number
# Commitment number on the opening channel 
commitment_number = 0
# obscured commitment number is result of xor operation 
commitment_number_obscured = lower48_to_obscure ^ commitment_number
# Extract the upper 24 bits of the obscured commitment number
upper_24_bits = (commitment_number_obscured >> 24) & 0xFFFFFF
# Combine the upper 8 bits (0x80) with the lower 24 bits (upper 24 of obscured number)
sequence = (0x80 << 24) | upper_24_bits
# Convert to bytes (byte, big-endian)
sequence = sequence.to_bytes(4, byteorder='big')
# Convert to little-endian 
sequence = sequence[::-1]

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

print("Commitment Number Osbscured", hex(commitment_number_obscured))

Commitment Number Osbscured 0xb433fd43a66f


### The Outputs

The Basis of Lightning Technology ([BOLT 3](https://github.com/lightning/bolts/blob/master/03-transactions.md)) defines the outputs as following:

* For every offered HTLC, if it is not trimmed, add an offered HTLC output.
* For every received HTLC, if it is not trimmed, add an received HTLC output.
* If the to_local amount is greater or equal to dust_limit_satoshis, add a to_local output.
* If the to_remote amount is greater or equal to dust_limit_satoshis, add a to_remote output.
* If option_anchors applies to the commitment transaction:
    * if to_local exists or there are untrimmed HTLCs, add a to_local_anchor output
    * if to_remote exists or there are untrimmed HTLCs, add a to_remote_anchor output

Since this is Alice’s first commitment transaction, it will have no outputs to Bob. That’s because Alice is the funder of the channel and isn't sending any sats to Bob initially. If she wanted to transfer funds at channel opening, she could have set the `push_msat` field in the `open_channel` message, specifying the amount in millisatoshis.

Additionally, because the channel hasn’t been opened yet, there are no offered or received HTLCs — so no HTLC outputs will be created either.

As a result, the outputs in the first commitment transaction will be as follows:

Alice first Commitment Transaction will have two outputs:
* to_local_anchor_output
* to_local_output

The Basis of Lightning Technology ([BOLT 3](https://github.com/lightning/bolts/blob/master/03-transactions.md)) specifies that transaction outputs must be sorted by value, from smallest to largest. Because of this, the to_local_anchor_output appears first — as it has a fixed amount of 330 sats, which corresponds to the default dust limit for P2WSH outputs.

**Anchor outputs** exists to prevent a malicious peer from attaching low-fee child transactions, which could block the victim from getting the commitment transaction confirmed in time. This defense was made possible by a change introduced in Bitcoin Core 0.19: [bitcoin/bitcoin#15681](https://github.com/bitcoin/bitcoin/pull/15681). It’s also the reason why all non-anchor outputs in the commitment transaction are CSV (CheckSequenceVerify)-locked: to delay their spendability and allow time for fee bumping via the anchor. The anchor outputs feature is optional and can be enabled only if both channel peers support it. However, starting with [Eclair v0.11.0](https://github.com/ACINQ/eclair/blob/master/docs/release-notes/eclair-v0.11.0.md) stop accepting channels that don't support anchor outputs.

A **trimmed HTLC** is an HTLC that does not get included in the commitment transaction because its value is too low to be economically spent. Specifically, if the value of the HTLC is below the dust limit plus the estimated fee required to claim it, the output is trimmed — in other words, left out of the transaction entirely.

This mechanism helps avoid bloating the commitment transaction with outputs that would either be unspendable or cost more in fees than they're worth.


### Alice First Commitment Transaction Outputs

#### The Local Anchor Output

This is the output that the local party, will be able to use to CPFP the commitment transaction and it is spendable by anyone after 16 blocks.

    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^
                   |   
         +----------------------+
         | P(local_delayed) + T |
         +----------------------+
                              ^
                              |  
                       +-----------+
                       | T = t * G |
                       +-----------+     
                              ^                          
                              |
                              +---+   +----------------------------------------------------------+
                              | t | = | TaggedHash ("Taptweak", P(local_delayed) || script_root) |
                              +---+   +----------------------------------------------------------+
                                                                                       ^
                                                                                       |
                                                                                     +---+
                                                                                     | h |
                                                                                     +---+
                                                                                       ^
                                                                                       |
                                                                                  +--------------+
                                                                                  | OP_16 OP_CSV |
                                                                                  +--------------+

#### To Local Output

The to_local output is responsible for paying the local peer their channel balance. The output must be revocable by the remote party at all times and only after to_self_delay blocks should the local party be able to spend from the output.

As you can see in the diagram below, both these paths are added as Taproot leaves in the Taproot tree and a public [NUMS](https://github.com/lightninglabs/lightning-node-connect/tree/master/mailbox/numsgen) (Nothing Up My Sleeve) point is used as the internal key which effectively cancels out the key-spend path.


    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^
                   |   +----------+
                    ---| NUMS + T |
                       +----------+
                                ^
                                |
                          +-----------+        
                          | T = t * G |
                          +-----------+        
                                ^
                                |
     +---+   +----------------------------------------------+
     | t | = | TaggedHash ("Taptweak", NUMS || script_root) |
     +---+   +----------------------------------------------+
                                                    ^
                                                    |
                                                 +-----+
                          ---------------------> | hAB |<------------------
                         |                       +-----+                   |
                         |                                                 |
                         |                                                 |
                      +----+                                            +----+
                      | hA |                                            | hB |
                      +----+                                            +----+
                         ^                                                 ^
                         |                                                 |
      +------------------------------+                       +---------------------------+
      | P(local_delayed) OP_CHECKSIG |                       | P(local_delayed) OP_DROP  |
      |                              |                       |                           |
      | to_self_delay OP_CSV OP_DROP |                       | P(revocation) OP_CHECKSIG |
      +------------------------------+                       +---------------------------+

#### Key Derivations

Each commitment transaction uses unique keys: `localpubkey`, `local_htlcpubkey`, `remote_htlcpubkey`, `local_delayedpubkey`, and `remote_delayedpubkey`. These public keys are derived by simply adding a per-commitment point to their respective base points. As defined at Basis of Lightning Technology ([BOLT 3](https://github.com/lightning/bolts/blob/master/03-transactions.md#key-derivation)):

```
pubkey = basepoint + SHA256(per_commitment_point || basepoint) * G

```

- The `localpubkey` uses the local node's `payment_basepoint`;
- The `local_htlcpubkey` uses the local node's `htlc_basepoint`;
- The `remote_htlcpubkey` uses the remote node's `htlc_basepoint`;
- The `local_delayedpubkey` uses the local node's `delayed_payment_basepoint`;
- The `remote_delayedpubkey` uses the remote node's `delayed_payment_basepoint`.

The `revocationpubkey` is a blinded public key. When the local node prepares a new commitment transaction for the remote node, it derives the `revocationpubkey` by combining its own `revocation_basepoint` with the remote node’s `per_commitment_point`.

```
revocationpubkey = revocation_basepoint * SHA256(revocation_basepoint || per_commitment_point) + per_commitment_point * SHA256(per_commitment_point || revocation_basepoint)
```

Later, when the remote node revokes that commitment by revealing the corresponding `per_commitment_secret`, the local node can derive the `revocationprivkey`. At that point, it possesses both secrets needed for the derivation: the `revocation_basepoint_secret` and the `per_commitment_secret`.

This construction ensures that neither the node providing the `basepoint` nor the node providing the `per_commitment_point` can derive the private key on their own—each requires the other’s secret.

```
revocationprivkey = revocation_basepoint_secret * SHA256(revocation_basepoint || per_commitment_point) + per_commitment_secret * SHA256(per_commitment_point || revocation_basepoint)
```

In [4]:
# Create Alice per-commitment
alice_per_commitment = per_commitment(alice_per_commitment_seed, commitment_number)
# Create Bob per-commitment
bob_per_commitment = per_commitment(bob_per_commitment_seed, commitment_number)

# Create Alice Delayed Public Key
alice_delayed_pubkey = derivate_key(alice_node_seed, family=4, channel_index=0).get_pubkey(alice_per_commitment.get_pub())
print(f"Alice Delayed PubKey: {alice_delayed_pubkey.get_bytes(bip340=False).hex()}")

# Create Bob Revocation Public Key
bob_revocation_pubkey = derivate_revocation_key(bob_node_seed, channel_index=0).get_pubkey(alice_per_commitment.get_pub())
print(f"Bob Revocation PubKey: {bob_revocation_pubkey.get_bytes(bip340=False).hex()}")

# Outputs for Alice First Commitment Transaction
# 0x02 outputs
output_count = bytes.fromhex("02")

# ANCHOR OUTPUT
# Anchor output amount
anchor_output_value_satoshis = 330
anchor_output_value = anchor_output_value_satoshis.to_bytes(8, byteorder="little", signed=False)

# OP_16 OP_CSV
script = CScript([OP_16, OP_CHECKSEQUENCEVERIFY])

# Method: ser_string(data) is a function which adds compactsize to input data.
hash_input = TAPSCRIPT_VER + ser_string(script)

# Anchor Output script_root
script_root = tagged_hash("TapBranch", hash_input)

# Anchor Output Tagged Hash
taptweak = tagged_hash("TapTweak", alice_delayed_pubkey.get_bytes() + script_root)
alice_delayed_pubkey_tweaked = alice_delayed_pubkey.tweak_add(taptweak)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
alice_anchor_spk = bytes.fromhex("51") + varint_len(alice_delayed_pubkey_tweaked.get_bytes()) + alice_delayed_pubkey_tweaked.get_bytes()

Alice Delayed PubKey: 03d6363615b3d00361158c0f48a4ef81ea12e214e8d56e24098759f6b4267dca8b
Bob Revocation PubKey: 025460add4edf0961bfcc6288be0241114d7d90005db28753d835f2353f1529710


### Fee Calculation

The fee calculation for commitment transactions  is based on the current `feerate_per_kw` sent by Alice into the `open_channel` message and the **expected** weight of the transaction.

In [13]:
# First commitment expected weight 
commitment_weight = taproot_commit_weight(NumTapOut=1, NumAnchorOut=1)
print("First Commitment Transaction Expected Weight:", commitment_weight)

# From open_channel and update_feerate messages (1 sat/vb = 250 sat/kw)
feerate_per_kw = 250

# Calculate the fee for the first commitment transaction
commitment_fee = int(commitment_weight * feerate_per_kw / 1000)

First Commitment Transaction Expected Weight: 624


In [14]:
# TO LOCAL OUTPUT
# to local output amount
to_local_value_sat = channel_value_sat - anchor_output_value_satoshis - commitment_fee
to_local_value = to_local_value_sat.to_bytes(8, byteorder="little", signed=False)

# Create the leaf scripts A and B
# P(local_delayed) OP_CHECKSIG to_self_delay OP_CSV OP_DROP
to_self_delay = 144
scriptA = CScript([alice_delayed_pubkey.get_bytes(), OP_CHECKSIG, to_self_delay.to_bytes(2,"big"), OP_CHECKSEQUENCEVERIFY, OP_DROP])

# P(local_delayed) OP_DROP P(revocation) OP_CHECKSIG
scriptB = CScript([alice_delayed_pubkey.get_bytes(), OP_DROP, bob_revocation_pubkey.get_bytes(), OP_CHECKSIG])

# Compute TapLeaves A and B
# Method: ser_string(data) is a function which adds compactsize to input data.
hash_inputA = TAPSCRIPT_VER + ser_string(scriptA)
hash_inputB = TAPSCRIPT_VER + ser_string(scriptB)
taggedhash_leafA = tagged_hash("TapLeaf", hash_inputA)
taggedhash_leafB = tagged_hash("TapLeaf", hash_inputB)

# Method: Returns tapbranch hash. Child hashes are lexographically sorted and then concatenated.
# l: tagged hash of left child
# r: tagged hash of right child
def tapbranch_hash(l, r):
    return tagged_hash("TapBranch", b''.join(sorted([l,r])))

# Compute Internal node TapBranch AB
root_ab = tapbranch_hash(taggedhash_leafA, taggedhash_leafB)

# Compute TapTweak
taptweak = tagged_hash("TapTweak", NUMS.get_bytes() + root_ab)
NUMS_tweaked = NUMS.tweak_add(taptweak)

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
alice_to_local_spk = bytes.fromhex("51") + varint_len(NUMS_tweaked.get_bytes()) + NUMS_tweaked.get_bytes()

outputs = (
    anchor_output_value
    + varint_len(alice_anchor_spk)
    + alice_anchor_spk
    + to_local_value
    + varint_len(alice_to_local_spk)
    + alice_to_local_spk
)

# Locktime: upper 8 bits are 0x20, lower 24 bits are the lower 24 bits of the obscured commitment number
# Extract the lower 24 bits of the obscured commitment number
lower_24_bits = commitment_number_obscured & 0xFFFFFF
# Combine the upper 8 bits (0x20) with the lower 24 bits (lower 24 of obscured number)
locktime = (0x20 << 24) | lower_24_bits
# Convert to bytes (1 byte, big-endian)
locktime = locktime.to_bytes(4, byteorder='big')
locktime = locktime[::-1]

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

unsigned_tx:  0200000001d19e49424a8302681e0a36620abd97c7003f828450c1702eb17ad4a82131db360000000000fd33b480024a01000000000000225120720dc4141fc966b2dcc95fb104ed557d2c055ba1b104de4c68ebb9a506acdd325a400f0000000000225120a52421c04d821c008f15346cb655de2b7be6934ed692b4966e2d9ac13e2a8ebf6fa64320
{
  "txid": "59c9832d781d1a7d370d67e5b72d60f3f3664672af89a29c2b668faf80b18a92",
  "hash": "59c9832d781d1a7d370d67e5b72d60f3f3664672af89a29c2b668faf80b18a92",
  "version": 2,
  "size": 137,
  "vsize": 137,
  "weight": 548,
  "locktime": 541304431,
  "vin": [
    {
      "txid": "36db3121a8d47ab12e70c15084823f00c797bd0a62360a1e6802834a42499ed1",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 2159293437
    }
  ],
  "vout": [
    {
      "value": "0.00000330",
      "n": 0,
      "scriptPubKey": {
        "asm": "1 720dc4141fc966b2dcc95fb104ed557d2c055ba1b104de4c68ebb9a506acdd32",
        "desc": "rawtr(720dc4141fc966b2dcc95fb104ed557d2c055ba1b104de4c

## The sighash for the key path spend

This is the “Schnorr key spend” case: you’re proving knowledge of the (tweaked) internal private key, with no script branch revealed. The message preimage is called msg_digest in [BIP-341](https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki).

In [15]:
index_of_this_input = bytes.fromhex("0000 0000")

# SIGHASH for key path spend
# See BIP-341 for details
sighash_epoch = bytes.fromhex("00")

# Control
hash_type = bytes.fromhex("00") # SIGHASH_DEFAULT (a new sighash type meaning implied SIGHASH_ALL)

# Transaction data
sha_prevouts = sha256(txid + index).digest()

sha_amounts = sha256(channel_value).digest()

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
sha_scriptpubkeys = sha256(
    varint_len(channel_spk)
    + channel_spk
).digest()

sha_sequences = sha256(sequence).digest()

sha_outputs = sha256(outputs).digest()

# Data about this input
spend_type = bytes.fromhex("00") # no annex present

sig_msg = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + sha_prevouts
    + sha_amounts
    + sha_scriptpubkeys
    + sha_sequences
    + sha_outputs
    + spend_type
    + index_of_this_input
)

tag_hash = sha256("TapSighash".encode()).digest()
sighash = sha256(tag_hash + tag_hash + sig_msg).digest()


## Signing the sighash

In [16]:
# Build participants to sort them using the same rule used in pubkeys aggregation.
participants = [
    (pubkey_alice_musig2.get_bytes(bip340=False), privkey_alice_musig2.get_bytes(), alice_per_commitment_seed),
    (pubkey_bob_musig2.get_bytes(bip340=False),   privkey_bob_musig2.get_bytes(), bob_per_commitment_seed),
]

# Reorder participants to match the sorted pubkeys
pk_to_tuple = {pk: (pk, sk, pcs) for pk, sk, pcs in participants}
participants = [pk_to_tuple[pk] for pk in sorted_pubkeys]

# Alice and Bob generates its own nonce pair (secnonce, pubnonce)
secnonces, pubnonces = [], []
for pk, sk, pcs in participants:
    # Use per-commitment nonce for Alice to get deterministic results
    if pk == pubkey_alice_musig2.get_bytes(bip340=False):
        sec, pub = nonce_per_commitment(pcs, commitment_number, sk, pk, agg_pubkey_tweaked, sighash)
    else:
        sec, pub = nonce_gen(sk, pk, agg_pubkey_tweaked, sighash, None)
    secnonces.append(sec)
    pubnonces.append(pub)

# Alice and Bob has exchanged already the pubnonces, so they can agregate them and create the session context
agg_nonce = nonce_agg(pubnonces)
session = SessionContext(agg_nonce, sorted_pubkeys, [bip86_tweak], [True], sighash)

# Alice and Bob produces their partial signatures
psigs = [sign(sec, sk, session) for sec, (_, sk, _) in zip(secnonces, participants)]

# Each side verify the other’s partial signature before proceeding
for i, psig in enumerate(psigs):
    assert partial_sig_verify(psig, pubnonces, sorted_pubkeys, [bip86_tweak], [True], sighash, i)

# Each side combines partial signatures into the final Schnorr signature
agg_sig = partial_sig_agg(psigs, session)

# Sanity check: verify with BIP340 against the *tweaked* x-only key ---
ok = schnorr_verify(sighash, agg_pubkey_tweaked, agg_sig)
print("aggregated Schnorr verifies?", ok)


aggregated Schnorr verifies? True


## The signed transaction

In [17]:
witness = (
    bytes.fromhex("01") # one stack item in the witness
    + varint_len(agg_sig)
    + agg_sig
)

# the final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed tx: ",signed_tx.hex())
# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(signed_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

print(node.testmempoolaccept(rawtxs=[signed_tx.hex()]))

signed tx:  02000000000101d19e49424a8302681e0a36620abd97c7003f828450c1702eb17ad4a82131db360000000000fd33b480024a01000000000000225120720dc4141fc966b2dcc95fb104ed557d2c055ba1b104de4c68ebb9a506acdd325a400f0000000000225120a52421c04d821c008f15346cb655de2b7be6934ed692b4966e2d9ac13e2a8ebf014044be92a57c0d82a6d43bafd246ab592d3fab1d8144eeac6531657555ede23890dbdbab2204b6817d985fc38970df8760478d3a663aa87355c1e4a38667db2d106fa64320
{
  "txid": "59c9832d781d1a7d370d67e5b72d60f3f3664672af89a29c2b668faf80b18a92",
  "hash": "5e5c0874beedc97913ecc94de1f382daf8cffbc56f488fbcf5571a7337935da9",
  "version": 2,
  "size": 205,
  "vsize": 154,
  "weight": 616,
  "locktime": 541304431,
  "vin": [
    {
      "txid": "36db3121a8d47ab12e70c15084823f00c797bd0a62360a1e6802834a42499ed1",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "44be92a57c0d82a6d43bafd246ab592d3fab1d8144eeac6531657555ede23890dbdbab2204b6817d985fc38970df8760478d3a663aa87355c1e